In [63]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd
from datetime import datetime, timedelta

# Setup Chrome WebDriver
options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the page
driver.get("https://fbref.com/en/squads/206d90db/2024-2025/all_comps/Barcelona-Stats-All-Competitions")
time.sleep(5)  # wait for content to load

match_data = []
rows = driver.find_elements(By.CSS_SELECTOR, "table.stats_table tbody tr")
for row in rows:
    try:
        # Get the match link from the date cell
        date_cell = row.find_element(By.CSS_SELECTOR, 'th[data-stat="date"] a')
        match_url = date_cell.get_attribute('href')
        match_date = date_cell.text.strip()
        # Try to get the time (can be missing)
        try:
            time_cell = row.find_element(By.CSS_SELECTOR, 'td[data-stat="start_time"] span.venuetime')
            match_time = time_cell.text.strip()
        except:
            match_time = None
        # Combine date and time into a datetime string
        if match_time:
            datetime_str = f"{match_date} {match_time}"
        else:
            datetime_str = match_date
        match_data.append({
            "url": match_url,
            "datetime": datetime_str
        })
    except:
        continue  # skip if date or link not found
# Click the "Previous Season" button if it exists
try:
    prev_button = driver.find_element(By.CSS_SELECTOR, "div.prevnext a")
    prev_url = prev_button.get_attribute("href")
    if prev_url:
        print("\nGoing to previous season:", prev_url)
        driver.get(prev_url)
        time.sleep(3)
        # --- Repeat scraping logic here for the previous season ---
        rows = driver.find_elements(By.CSS_SELECTOR, "table.stats_table tbody tr")
        for row in rows:
            try:
                date_cell = row.find_element(By.CSS_SELECTOR, 'th[data-stat="date"] a')
                match_url = date_cell.get_attribute('href')
                match_date = date_cell.text.strip()
                try:
                    time_cell = row.find_element(By.CSS_SELECTOR, 'td[data-stat="start_time"] span.venuetime')
                    match_time = time_cell.text.strip()
                except:
                    match_time = None
                datetime_str = f"{match_date} {match_time}" if match_time else match_date
                match_data.append({
                    "url": match_url,
                    "datetime": datetime_str
                })
            except:
                continue
except:
    print("No 'Previous Season' button found.")
rows = driver.find_elements(By.CSS_SELECTOR, "table.stats_table tbody tr")
for row in rows:
    try:
        # Get the match link from the date cell
        date_cell = row.find_element(By.CSS_SELECTOR, 'th[data-stat="date"] a')
        match_url = date_cell.get_attribute('href')
        match_date = date_cell.text.strip()
        # Try to get the time (can be missing)
        try:
            time_cell = row.find_element(By.CSS_SELECTOR, 'td[data-stat="start_time"] span.venuetime')
            match_time = time_cell.text.strip()
        except:
            match_time = None
        # Combine date and time into a datetime string
        if match_time:
            datetime_str = f"{match_date} {match_time}"
        else:
            datetime_str = match_date
        match_data.append({
            "url": match_url,
            "datetime": datetime_str
        })
    except:
        continue  # skip if date or link not found


time.sleep(3)

# Close browser
driver.quit()

# Convert to DataFrame
df = pd.DataFrame(match_data)

# Rename columns if needed
df = df.rename(columns={"datetime": "date", "url": "link"})

df = df.drop_duplicates().sort_values('date', ascending=False).head(100)
df.to_csv("barcelona_matches.csv", index=False)


Going to previous season: https://fbref.com/en/squads/206d90db/2023-2024/Barcelona-Stats
